In [70]:
import pandas as pd
import numpy as np
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import matplotlib.pyplot as plt

In [71]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

chrome_options = Options()
chrome_options.add_argument('--headless')  # Run in background
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-notifications')

driver = webdriver.Chrome(options=chrome_options)

url = "https://www.fundsexplorer.com.br/ranking"
driver.get(url)
time.sleep(5)

html = driver.page_source
df = pd.read_html(html)[0]

driver.quit()

print("Data successfully scraped!")
print(df.head())

/tmp/ipykernel_20678/2965704421.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html)[0]


Data successfully scraped!
   Fundos       Setor Liquidez Diária (R$)  P/VP Dividend Yield  \
0  AAGR11  Indefinido             6.094,60   NaN         1,03 %   
1  AAZQ11  Indefinido           660.235,10  70.0         1,35 %   
2  ABCP11   Shoppings            61.212,35  70.0         0,90 %   
3  AFHI11      Papéis         1.118.388,50  94.0         1,07 %   
4  AGRX11      Outros           362.544,25   NaN         0,81 %   

  Variação Preço Patrimônio Líquido Num. Cotistas  
0        -1,50 %                NaN             0  
1         4,31 %     208.745.497,59        29.829  
2         2,18 %     503.496.718,25        15.558  
3        -5,29 %     432.224.228,96        42.539  
4       -20,22 %                NaN             0  


In [72]:
df = pd.read_csv('/home/wvmwill/enviroment/analytics-fii/funds_explorer_data.csv')

df.head()

sectors = df['Setor'].unique()
contagem = df['Setor'].value_counts()
print(f"Quantidade de Setores: {len((sectors))}")
print(contagem)

Quantidade de Setores: 23
Setor
Indefinido                          116
Papéis                               94
Lajes Corporativas                   60
Misto                                44
Imóveis Industriais e Logísticos     44
Fundo de Fundos                      35
Fundo de Desenvolvimento             35
Shoppings                            29
Imóveis Residenciais                 10
Imóveis Comerciais - Outros           7
Hospitalar                            6
Educacional                           5
Varejo                                5
Agências de Bancos                    5
Outros                                4
Serviços Financeiros Diversos         4
Hotéis                                4
Incorporações                         2
Logística                             2
Fiagro                                1
Agricultura                           1
Tecidos. Vestuário e Calçados         1
Name: count, dtype: int64


In [73]:
df.isna().sum()

Fundos                   0
Setor                    1
Liquidez Diária (R$)    47
P/VP                    71
Dividend Yield           3
Variação Preço           3
Patrimônio Líquido      46
Num. Cotistas            0
dtype: int64

In [74]:
data_categoricals = ['Fundos', 'Setor']
idx = df[df['Setor'].isna()].index

In [75]:
df.drop(idx, inplace=True)

In [76]:
df[data_categoricals].isna().sum()


df[data_categoricals] = df[data_categoricals].astype('category')

df.dtypes

Fundos                  category
Setor                   category
Liquidez Diária (R$)      object
P/VP                     float64
Dividend Yield            object
Variação Preço            object
Patrimônio Líquido        object
Num. Cotistas             object
dtype: object

In [77]:
col_floats = list(df.iloc[:,2:-1].columns)

In [78]:
df[col_floats] = df[col_floats].fillna(value=0)
df[col_floats].head()

,Liquidez Diária (R$),P/VP,Dividend Yield,Variação Preço,Patrimônio Líquido
0,"5.848,90",0.0,"1,03 %","-1,50 %",0
1,"654.462,24",70.0,"1,35 %","4,31 %","208.745.497,59"
2,"60.839,95",70.0,"0,90 %","2,18 %","503.496.718,25"
3,"1.111.830,95",94.0,"0,00 %","0,00 %","432.224.228,96"
4,"383.579,90",0.0,"0,81 %","-20,22 %",0


In [79]:
df[col_floats].isna().sum()

Liquidez Diária (R$)    0
P/VP                    0
Dividend Yield          0
Variação Preço          0
Patrimônio Líquido      0
dtype: int64

In [80]:
df[col_floats]

,Liquidez Diária (R$),P/VP,Dividend Yield,Variação Preço,Patrimônio Líquido
0,"5.848,90",0.0,"1,03 %","-1,50 %",0
1,"654.462,24",70.0,"1,35 %","4,31 %","208.745.497,59"
2,"60.839,95",70.0,"0,90 %","2,18 %","503.496.718,25"
3,"1.111.830,95",94.0,"0,00 %","0,00 %","432.224.228,96"
4,"383.579,90",0.0,"0,81 %","-20,22 %",0
...,...,...,...,...,...
510,0,0.0,"0,00 %","0,00 %",0
511,"1.353,76",140.0,"0,54 %","0,70 %","42.783.877,65"
512,277,103.0,"1,24 %","16,56 %","31.110.512,89"
513,"204.552,71",64.0,"1,11 %","-1,39 %","154.014.555,33"


In [81]:
df['Liquidez Diária (R$)'] = (df['Liquidez Diária (R$)']
    .apply(lambda x: str(x).replace('.', '').replace(',', '.'))
    .astype(float))

In [82]:
df['Dividend Yield'] = df['Dividend Yield'].apply(lambda x: str(x).replace('%', '').replace(',','.')).astype(float)
df['Dividend Yield'] = df['Dividend Yield'].div(100)
df.head()

,Fundos,Setor,Liquidez Diária (R$),P/VP,Dividend Yield,Variação Preço,Patrimônio Líquido,Num. Cotistas
0,AAGR11,Indefinido,5848.90,0.0,0.0103,"-1,50 %",0,0
1,AAZQ11,Indefinido,654462.24,70.0,0.0135,"4,31 %","208.745.497,59",29.829
2,ABCP11,Shoppings,60839.95,70.0,0.0090,"2,18 %","503.496.718,25",15.558
3,AFHI11,Papéis,1111830.95,94.0,0.0000,"0,00 %","432.224.228,96",42.539
4,AGRX11,Outros,383579.90,0.0,0.0081,"-20,22 %",0,0


In [83]:
df['Variação Preço'] = df['Variação Preço'].apply(lambda x: str(x).replace('%', '').replace(',','.')).astype(float)
df['Variação Preço'] = df['Variação Preço'].div(100)

In [84]:
df['Num. Cotistas'] = df['Num. Cotistas'].apply(lambda x: str(x).replace('.', '')).astype(int)

In [85]:
df['Patrimônio Líquido'] = (df['Patrimônio Líquido']
    .apply(lambda x: str(x).replace('.', '').replace(',', '.'))
    .astype(float))

In [86]:
df.dtypes

df.to_csv('fiis.csv')

In [87]:
df.describe()

,Liquidez Diária (R$),P/VP,Dividend Yield,Variação Preço,Patrimônio Líquido,Num. Cotistas
count,5.140000e+02,514.000000,514.000000,514.000000,5.140000e+02,5.140000e+02
mean,9.260822e+05,81.214008,0.020914,-0.026944,4.484046e+08,2.847373e+04
std,2.035667e+06,150.528843,0.075851,0.097057,8.589898e+08,8.389792e+04
min,0.000000e+00,0.000000,0.000000,-0.902800,-1.968309e+07,0.000000e+00
25%,1.063347e+04,49.000000,0.005000,-0.049950,6.090114e+07,1.385000e+02
50%,1.442823e+05,76.000000,0.009600,-0.010150,1.471627e+08,3.538000e+03
75%,7.514995e+05,93.000000,0.011900,0.000000,4.111710e+08,1.393400e+04
max,1.696072e+07,2382.000000,0.868400,0.738300,7.440193e+09,1.220123e+06


In [88]:
df.columns

Index(['Fundos', 'Setor', 'Liquidez Diária (R$)', 'P/VP', 'Dividend Yield',
       'Variação Preço', 'Patrimônio Líquido', 'Num. Cotistas'],
      dtype='object')

In [89]:
key_indicators = [
    'Fundos',
    'Setor',
    'Dividend Yield',
    'P/VP'
]

df_fiis = df[key_indicators]

In [90]:
media_setor = df_fiis.groupby('Setor')['Dividend Yield'].agg(['mean','std', 'count'])

media_setor.sort_values(by='mean', ascending=False)

/tmp/ipykernel_20678/3006660759.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  media_setor = df_fiis.groupby('Setor')['Dividend Yield'].agg(['mean','std', 'count'])


,mean,std,count
Setor,,,
Educacional,0.035880,0.065050,5
Fundo de Fundos,0.032563,0.091076,35
Tecidos. Vestuário e Calçados,0.030300,NaN,1
Fundo de Desenvolvimento,0.029117,0.055010,35
Varejo,0.028680,0.038608,5
Indefinido,0.024945,0.097875,116
Papéis,0.024127,0.099012,94
Lajes Corporativas,0.022632,0.085594,60
Imóveis Residenciais,0.014600,0.026215,10


In [101]:
# Sort and select specific columns
order_by_found = df_fiis.sort_values(by='Dividend Yield', ascending=False)[['Fundos', 'Dividend Yield']].reset_index(drop=True)
order_by_found.head(10)

df_all = order_by_found.merge(df, on='Fundos', how='left')
df_all.drop(columns=['Dividend Yield_y'], inplace=True)
df_all.head(10)




,Fundos,Dividend Yield_x,Setor,Liquidez Diária (R$),P/VP,Variação Preço,Patrimônio Líquido,Num. Cotistas
0,LSPA11,0.8684,Papéis,8983.43,115.0,0.0000,8.415548e+06,707
1,IDFI11,0.7804,Indefinido,2592.80,125.0,-0.5434,3.283392e+06,617
2,BDIV11,0.5292,Indefinido,757044.62,0.0,-0.0282,0.000000e+00,0
3,NVRP11,0.5178,Indefinido,2603.18,0.0,0.0000,0.000000e+00,0
4,MFAI11,0.4884,Fundo de Fundos,11326.82,97.0,0.0266,2.798860e+07,2301
5,HGPO11,0.4822,Lajes Corporativas,568563.24,78.0,0.0505,2.727971e+08,10975
6,BBFI11B,0.4751,Lajes Corporativas,182918.19,41.0,0.0422,1.120514e+08,7144
7,JBFO11,0.4382,Papéis,8200.00,20.0,-0.6716,9.686058e+07,366
8,PNPR11,0.2523,Fundo de Desenvolvimento,13424.62,78.0,0.0000,6.531829e+07,26
9,IDGR11,0.2379,Misto,1194.38,69.0,-0.3108,2.249712e+06,154
